출처: https://wikidocs.net/262595

## Agentic RAG

In [ ]:
from dotenv import load_dotenv
from langsmith import traceable
import os

load_dotenv()
os.environ["LANGCHAIN_PROJECT"] = "agents-rag"


In [ ]:
# TavilySearchResults 클래스를 langchain_community.tools.tavily_search 모듈에서 가져옵니다.
from langchain_community.tools.tavily_search import TavilySearchResults

# TavilySearchResults 클래스의 인스턴스를 생성합니다
# k=6은 검색 결과를 6개까지 가져오겠다는 의미입니다
search = TavilySearchResults(k=6)


In [ ]:
# 검색 결과를 가져옵니다.
search.invoke("판교 카카오 프렌즈샵 아지트점의 전화번호는 무엇인가요?")


실습에 활용한 문서

소프트웨어정책연구소(SPRi) - 2023년 12월호

- 저자: 유재흥(AI정책연구실 책임연구원), 이지수(AI정책연구실 위촉연구원)
- 링크: https://spri.kr/posts/view/23669
- 파일명: SPRI_AI_Brief_2023년12월호_F.pdf

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")

# 텍스트 분할기를 사용하여 문서를 분할합니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 문서를 로드하고 분할합니다.
split_docs = loader.load_and_split(text_splitter)

# VectorStore를 생성합니다.
vector = FAISS.from_documents(split_docs, OpenAIEmbeddings())

# Retriever를 생성합니다.
retriever = vector.as_retriever()


In [ ]:
# 문서에서 관련성 높은 문서를 가져옵니다.
retriever.invoke("삼성전자가 개발한 생성형 AI 관련 내용을 문서에서 찾아줘")


In [ ]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",  # 도구의 이름을 입력합니다.
    description="use this tool to search information from the PDF document",  # 도구에 대한 설명을 자세히 기입해야 합니다!!
)


In [ ]:
# tools 리스트에 search와 retriever_tool을 추가합니다.
tools = [search, retriever_tool]


## Agent 생성

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# LLM 정의
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Prompt 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `pdf_search` tool for searching information from the PDF document. "
            "If you can't find the information from the PDF document, use the `search` tool for searching information from the web.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)


In [ ]:
from langchain.agents import create_tool_calling_agent

# tool calling agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)


In [ ]:
from langchain.agents import AgentExecutor

# AgentExecutor 생성
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)


In [ ]:
from langchain_teddynote.messages import AgentStreamParser

# 각 단계별 출력을 위한 파서 생성
agent_stream_parser = AgentStreamParser()


In [ ]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
result = agent_executor.stream(
    {"input": "2024년 프로야구 플레이오프 진출한 5개 팀을 검색하여 알려주세요."}
)

for step in result:
    print(step)
    # # 중간 단계를 parser 를 사용하여 단계별로 출력
    # agent_stream_parser.process_agent_steps(step)


In [ ]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
result = agent_executor.stream(
    {"input": "삼성전자가 자체 개발한 생성형 AI 관련된 정보를 문서에서 찾아주세요."}
)

for step in result:
    # 중간 단계를 parser 를 사용하여 단계별로 출력
    agent_stream_parser.process_agent_steps(step)


## 이전 대화내용 기억하는 Agent

이전의 대화내용을 기억하기 위해서는 RunnableWithMessageHistory 를 사용하여 AgentExecutor 를 감싸줍니다.

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# session_id 를 저장할 딕셔너리 생성
store = {}


# session_id 를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in store:  # session_id 가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 채팅 메시지 기록이 추가된 에이전트를 생성합니다.
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # 대화 session_id
    get_session_history,
    # 프롬프트의 질문이 입력되는 key: "input"
    input_messages_key="input",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
)


In [ ]:
# 질의에 대한 답변을 스트리밍으로 출력 요청
response = agent_with_chat_history.stream(
    {"input": "삼성전자가 개발한 생성형 AI 관련된 정보를 문서에서 찾아주세요."},
    # session_id 설정
    config={"configurable": {"session_id": "abc123"}},
)

# 출력 확인
for step in response:
    agent_stream_parser.process_agent_steps(step)


In [ ]:
response = agent_with_chat_history.stream(
    {"input": "이전의 답변을 영어로 번역해 주세요."},
    # session_id 설정
    config={"configurable": {"session_id": "abc123"}},
)

# 출력 확인
for step in response:
    agent_stream_parser.process_agent_steps(step)


# Agent 템플릿

In [ ]:
from dotenv import load_dotenv
from langsmith import traceable
import os

load_dotenv()
os.environ["LANGCHAIN_PROJECT"] = "agents-rag"


# 필요한 모듈 import
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.vectorstores import FAISS
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.document_loaders import PyMuPDFLoader
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.runnables.history import RunnableWithMessageHistory
# from langchain_teddynote.messages import AgentStreamParser

########## 1. 도구를 정의합니다 ##########

### 1-1. Search 도구 ###
# TavilySearchResults 클래스의 인스턴스를 생성합니다
# k=6은 검색 결과를 6개까지 가져오겠다는 의미입니다
search = TavilySearchResults(k=6)

### 1-2. PDF 문서 검색 도구 (Retriever) ###
# PDF 파일 로드. 파일의 경로 입력
loader = PyMuPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")

# 텍스트 분할기를 사용하여 문서를 분할합니다.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 문서를 로드하고 분할합니다.
split_docs = loader.load_and_split(text_splitter)

# VectorStore를 생성합니다.
vector = FAISS.from_documents(split_docs, OpenAIEmbeddings())

# Retriever를 생성합니다.
retriever = vector.as_retriever()

retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",  # 도구의 이름을 입력합니다.
    description="use this tool to search information from the PDF document",  # 도구에 대한 설명을 자세히 기입해야 합니다!!
)

### 1-3. tools 리스트에 도구 목록을 추가합니다 ###
# tools 리스트에 search와 retriever_tool을 추가합니다.
tools = [search, retriever_tool]

########## 2. LLM 을 정의합니다 ##########
# LLM 모델을 생성합니다.
llm = ChatOpenAI(model="gpt-4o", temperature=0)

########## 3. Prompt 를 정의합니다 ##########

# Prompt 를 정의합니다 - 이 부분을 수정할 수 있습니다!
# Prompt 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. "
            "Make sure to use the `pdf_search` tool for searching information from the PDF document. "
            "If you can't find the information from the PDF document, use the `search` tool for searching information from the web.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

########## 4. Agent 를 정의합니다 ##########

# 에이전트를 생성합니다.
# llm, tools, prompt를 인자로 사용합니다.
agent = create_tool_calling_agent(llm, tools, prompt)

########## 5. AgentExecutor 를 정의합니다 ##########

# AgentExecutor 클래스를 사용하여 agent와 tools를 설정하고, 상세한 로그를 출력하도록 verbose를 True로 설정합니다.
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

########## 6. 채팅 기록을 수행하는 메모리를 추가합니다. ##########

# session_id 를 저장할 딕셔너리 생성
store = {}


# session_id 를 기반으로 세션 기록을 가져오는 함수
def get_session_history(session_ids):
    if session_ids not in store:  # session_id 가 store에 없는 경우
        # 새로운 ChatMessageHistory 객체를 생성하여 store에 저장
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]  # 해당 세션 ID에 대한 세션 기록 반환


# 채팅 메시지 기록이 추가된 에이전트를 생성합니다.
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # 대화 session_id
    get_session_history,
    # 프롬프트의 질문이 입력되는 key: "input"
    input_messages_key="input",
    # 프롬프트의 메시지가 입력되는 key: "chat_history"
    history_messages_key="chat_history",
)

########## 7. Agent 파서를 정의합니다. ##########
# agent_stream_parser = AgentStreamParser()


In [12]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input": "구글이 앤스로픽에 투자한 금액을 문서에서 찾아줘"},
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "abc123"}},
)

for step in response:
    print(step)
    # agent_stream_parser.process_agent_steps(step)


{'actions': [ToolAgentAction(tool='pdf_search', tool_input={'query': '구글 앤스로픽 투자 금액'}, log="\nInvoking: `pdf_search` with `{'query': '구글 앤스로픽 투자 금액'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_qS7ztL2n3wOv2ZXlDzGn5bN1', 'function': {'arguments': '{"query":"구글 앤스로픽 투자 금액"}', 'name': 'pdf_search'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_c7ca0ebaca'}, id='run-2e094cc2-16a5-425c-93af-c085f9975530', tool_calls=[{'name': 'pdf_search', 'args': {'query': '구글 앤스로픽 투자 금액'}, 'id': 'call_qS7ztL2n3wOv2ZXlDzGn5bN1', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'pdf_search', 'args': '{"query":"구글 앤스로픽 투자 금액"}', 'id': 'call_qS7ztL2n3wOv2ZXlDzGn5bN1', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_qS7ztL2n3wOv2ZXlDzGn5bN1')], 'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_qS

In [ ]:
{
    'actions': [
        ToolAgentAction(
            tool='pdf_search', 
            tool_input={'query': '구글 앤스로픽 투자 금액'}, 
            log="\nInvoking: `pdf_search` with `{'query': '구글 앤스로픽 투자 금액'}`\n\n\n", 
            message_log=[
                AIMessageChunk(
                    content='', 
                    additional_kwargs={
                        'tool_calls': [{
                            'index': 0, 
                            'id': 'call_qS7ztL2n3wOv2ZXlDzGn5bN1', 
                            'function': {
                                'arguments': '{"query":"구글 앤스로픽 투자 금액"}', 
                                'name': 'pdf_search'
                            }, 
                            'type': 'function'
                        }]
                    }, 
                    response_metadata={
                        'finish_reason': 'tool_calls', 
                        'model_name': 'gpt-4o-2024-08-06', 
                        'system_fingerprint': 'fp_c7ca0ebaca'
                    }, 
                    id='run-2e094cc2-16a5-425c-93af-c085f9975530', 
                    tool_calls=[{
                        'name': 'pdf_search', 
                        'args': {'query': '구글 앤스로픽 투자 금액'}, 
                        'id': 'call_qS7ztL2n3wOv2ZXlDzGn5bN1', 
                        'type': 'tool_call'
                    }], 
                    tool_call_chunks=[{
                        'name': 'pdf_search', 
                        'args': '{"query":"구글 앤스로픽 투자 금액"}', 
                        'id': 'call_qS7ztL2n3wOv2ZXlDzGn5bN1', 
                        'index': 0, 
                        'type': 'tool_call_chunk'
                    }]
                )
            ], 
            tool_call_id='call_qS7ztL2n3wOv2ZXlDzGn5bN1'
        )
    ], 
    'messages': [
        AIMessageChunk(
            content='', 
            additional_kwargs={
                'tool_calls': [{
                    'index': 0, 
                    'id': 'call_qS7ztL2n3wOv2ZXlDzGn5bN1', 
                    'function': {
                        'arguments': '{"query":"구글 앤스로픽 투자 금액"}', 
                        'name': 'pdf_search'
                    }, 
                    'type': 'function'
                }]
            }, 
            response_metadata={
                'finish_reason': 'tool_calls', 
                'model_name': 'gpt-4o-2024-08-06', 
                'system_fingerprint': 'fp_c7ca0ebaca'
            }, 
            id='run-2e094cc2-16a5-425c-93af-c085f9975530', 
            tool_calls=[{
                'name': 'pdf_search', 
                'args': {'query': '구글 앤스로픽 투자 금액'}, 
                'id': 'call_qS7ztL2n3wOv2ZXlDzGn5bN1', 
                'type': 'tool_call'
            }], 
            tool_call_chunks=[{
                'name': 'pdf_search', 
                'args': '{"query":"구글 앤스로픽 투자 금액"}', 
                'id': 'call_qS7ztL2n3wOv2ZXlDzGn5bN1', 
                'index': 0, 
                'type': 'tool_call_chunk'}]
        )
    ]
}


In [ ]:
{
    'steps': [
        AgentStep(
            action=ToolAgentAction(
                tool='pdf_search', 
                tool_input={'query': '구글 앤스로픽 투자 금액'}, 
                log="\nInvoking: `pdf_search` with `{'query': '구글 앤스로픽 투자 금액'}`\n\n\n", 
                message_log=[
                    AIMessageChunk(
                        content='', 
                        additional_kwargs={
                            'tool_calls': [{
                                'index': 0, 
                                'id': 'call_qS7ztL2n3wOv2ZXlDzGn5bN1', 
                                'function': {
                                    'arguments': '{"query":"구글 앤스로픽 투자 금액"}', 
                                    'name': 'pdf_search'
                                }, 
                                'type': 'function'}]
                        }, 
                        response_metadata={
                            'finish_reason': 'tool_calls', 
                            'model_name': 'gpt-4o-2024-08-06', 
                            'system_fingerprint': 'fp_c7ca0ebaca'
                        }, 
                        id='run-2e094cc2-16a5-425c-93af-c085f9975530', 
                        tool_calls=[{
                            'name': 'pdf_search', 
                            'args': {'query': '구글 앤스로픽 투자 금액'}, 
                            'id': 'call_qS7ztL2n3wOv2ZXlDzGn5bN1', 
                            'type': 'tool_call'
                        }], 
                        tool_call_chunks=[{
                            'name': 'pdf_search', 
                            'args': '{"query":"구글 앤스로픽 투자 금액"}', 
                            'id': 'call_qS7ztL2n3wOv2ZXlDzGn5bN1', 
                            'index': 0, 
                            'type': 'tool_call_chunk'
                        }]
                    )
                ], 
                tool_call_id='call_qS7ztL2n3wOv2ZXlDzGn5bN1'
            ), 
            observation='1. 정책/법제  \n2. 기업/산업 \n3. 기술/연구 \n 4. 인력/교육\n구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화 \nn 구글이 앤스로픽에 최대 20억 달러 투자에 합의하고 5억 달러를 우선 투자했으며, 앤스로픽은 \n구글과 클라우드 서비스 사용 계약도 체결\nn 3대 클라우드 사업자인 구글, 마이크로소프트, 아마존은 차세대 AI 모델의 대표 기업인 \n앤스로픽 및 오픈AI와 협력을 확대하는 추세\nKEY Contents\n£ 구글, 앤스로픽에 최대 20억 달러 투자 합의 및 클라우드 서비스 제공\nn 구글이 2023년 10월 27일 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며, 이 중 5억 \n달러를 우선 투자하고 향후 15억 달러를 추가로 투자할 방침\n∙구글은 2023년 2월 앤스로픽에 이미 5억 5,000만 달러를 투자한 바 있으며, 아마존도 지난 9월 \n앤스로픽에 최대 40억 달러의 투자 계획을 공개\n∙한편, 2023년 11월 8일 블룸버그 보도에 따르면 앤스로픽은 구글의 클라우드 서비스 사용을 위해 \n4년간 30억 달러 규모의 계약을 체결\n∙오픈AI 창업자 그룹의 일원이었던 다리오(Dario Amodei)와 다니엘라 아모데이(Daniela Amodei) \n남매가 2021년 설립한 앤스로픽은 챗GPT의 대항마 ‘클로드(Claude)’ LLM을 개발\nn 아마존과 구글의 앤스로픽 투자에 앞서, 마이크로소프트는 차세대 AI 모델의 대표 주자인 오픈\nAI와 협력을 확대\n∙마이크로소프트는 오픈AI에 앞서 투자한 30억 달러에 더해 2023년 1월 추가로 100억 달러를 \n투자하기로 하면서 오픈AI의 지분 49%를 확보했으며, 오픈AI는 마이크로소프트의 애저(Azure) \n클라우드 플랫폼을 사용해 AI 모델을 훈련\n£ 구글, 클라우드 경쟁력 강화를 위해 생성 AI 투자 확대\nn 구글은 수익률이 높은 클라우드 컴퓨팅 시장에서 아마존과 마이크로소프트를 따라잡고자 생성 AI를 \n통한 기업 고객의 클라우드 지출 확대를 위해 AI 투자를 지속\n\nn 구글은 수익률이 높은 클라우드 컴퓨팅 시장에서 아마존과 마이크로소프트를 따라잡고자 생성 AI를 \n통한 기업 고객의 클라우드 지출 확대를 위해 AI 투자를 지속  \n∙구글은 앤스로픽 외에도 AI 동영상 제작 도구를 개발하는 런웨이(Runway)와 오픈소스 소프트웨어 \n기업 허깅 페이스(Hugging Face)에도 투자\n∙구글은 챗GPT의 기반 기술과 직접 경쟁할 수 있는 차세대 LLM ‘제미니(Gemini)’를 포함한 자체 AI \n시스템 개발에도 수십억 달러를 투자했으며, 2024년 제미니를 출시할 계획\n☞ 출처 : The Wall Street Journal, Google Commits $2 Billion in Funding to AI Startup Anthropic, 2023.10.27.\nBloomberg, AI Startup Anthropic to Use Google Chips in Expanded Partnership, 2023.11.09.\n\n1. 정책/법제  \n2. 기업/산업 \n3. 기술/연구 \n 4. 인력/교육\n미국 프런티어 모델 포럼, 1,000만 달러 규모의 AI 안전 기금 조성\nn 구글, 앤스로픽, 마이크로소프트, 오픈AI가 참여하는 프런티어 모델 포럼이 자선단체와 함께 AI \n안전 연구를 위한 1,000만 달러 규모의 AI 안전 기금을 조성\nn 프런티어 모델 포럼은 AI 모델의 취약점을 발견하고 검증하는 레드팀 활동을 지원하기 위한 \n모델 평가 기법 개발에 자금을 중점 지원할 계획\nKEY Contents\n£ 프런티어 모델 포럼, 자선단체와 함께 AI 안전 연구를 위한 기금 조성\nn 구글, 앤스로픽, 마이크로소프트, 오픈AI가 출범한 프런티어 모델 포럼이 2023년 10월 25일 AI 안전 \n연구를 위한 기금을 조성한다고 발표\n∙참여사들은 맥거번 재단(Patrick J. McGovern Foundation), 데이비드 앤 루실 패커드 재단(The \nDavid and Lucile Packard Foundation) 등의 자선단체와 함께 AI 안전 연구를 위한 기금에 \n1,000만 달러 이상을 기부 \n∙또한 신기술의 거버넌스와 안전 분야에서 전문성을 갖춘 브루킹스 연구소 출신의 크리스 메서롤(Chris \nMeserole)을 포럼의 상무이사로 임명\nn 최근 AI 기술이 급속히 발전하면서 AI 안전에 관한 연구가 부족한 시점에, 포럼은 이러한 격차를 해소\n하기 위해 AI 안전 기금을 조성\n∙참여사들은 지난 7월 백악관 주재의 AI 안전 서약에서 외부자의 AI 시스템 취약점 발견과 신고를 \n촉진하기로 약속했으며, 약속을 이행하기 위해 기금을 활용해 외부 연구집단의 AI 시스템 평가에 \n자금을 지원할 계획\n£ AI 안전 기금으로 AI 레드팀을 위한 모델 평가 기법 개발을 중점 지원할 계획\nn 프런티어 모델 포럼은 AI 안전 기금을 통해 AI 레드팀 활동을 위한 새로운 모델 평가 기법의 개발을 \n중점 지원할 예정\n\n▹ 구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화 ················································ 11\n   ▹ IDC, 2027년 AI 소프트웨어 매출 2,500억 달러 돌파 전망··········································· 12\n   ▹ 빌 게이츠, AI 에이전트로 인한 컴퓨터 사용의 패러다임 변화 전망································ 13\n   ▹ 유튜브, 2024년부터 AI 생성 콘텐츠 표시 의무화···························································· 14\n 3. 기술/연구\n   ▹ 영국 과학혁신기술부, AI 안전 연구소 설립 발표······························································ 15\n   ▹ 구글 딥마인드, 범용 AI 모델의 기능과 동작에 대한 분류 체계 발표······························ 16\n   ▹ 갈릴레오의 LLM 환각 지수 평가에서 GPT-4가 가장 우수 ··········································· 17\n   \n 4. 인력/교육     \n   ▹ 영국 옥스퍼드 인터넷 연구소, AI 기술자의 임금이 평균 21% 높아······························· 18\n   \n   \n \nⅡ. 주요 행사\n   ▹CES 2024 ····························································································································· 19'
        )
    ], 
    'messages': [
        FunctionMessage(
            content='1. 정책/법제  \n2. 기업/산업 \n3. 기술/연구 \n 4. 인력/교육\n구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화 \nn 구글이 앤스로픽에 최대 20억 달러 투자에 합의하고 5억 달러를 우선 투자했으며, 앤스로픽은 \n구글과 클라우드 서비스 사용 계약도 체결\nn 3대 클라우드 사업자인 구글, 마이크로소프트, 아마존은 차세대 AI 모델의 대표 기업인 \n앤스로픽 및 오픈AI와 협력을 확대하는 추세\nKEY Contents\n£ 구글, 앤스로픽에 최대 20억 달러 투자 합의 및 클라우드 서비스 제공\nn 구글이 2023년 10월 27일 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며, 이 중 5억 \n달러를 우선 투자하고 향후 15억 달러를 추가로 투자할 방침\n∙구글은 2023년 2월 앤스로픽에 이미 5억 5,000만 달러를 투자한 바 있으며, 아마존도 지난 9월 \n앤스로픽에 최대 40억 달러의 투자 계획을 공개\n∙한편, 2023년 11월 8일 블룸버그 보도에 따르면 앤스로픽은 구글의 클라우드 서비스 사용을 위해 \n4년간 30억 달러 규모의 계약을 체결\n∙오픈AI 창업자 그룹의 일원이었던 다리오(Dario Amodei)와 다니엘라 아모데이(Daniela Amodei) \n남매가 2021년 설립한 앤스로픽은 챗GPT의 대항마 ‘클로드(Claude)’ LLM을 개발\nn 아마존과 구글의 앤스로픽 투자에 앞서, 마이크로소프트는 차세대 AI 모델의 대표 주자인 오픈\nAI와 협력을 확대\n∙마이크로소프트는 오픈AI에 앞서 투자한 30억 달러에 더해 2023년 1월 추가로 100억 달러를 \n투자하기로 하면서 오픈AI의 지분 49%를 확보했으며, 오픈AI는 마이크로소프트의 애저(Azure) \n클라우드 플랫폼을 사용해 AI 모델을 훈련\n£ 구글, 클라우드 경쟁력 강화를 위해 생성 AI 투자 확대\nn 구글은 수익률이 높은 클라우드 컴퓨팅 시장에서 아마존과 마이크로소프트를 따라잡고자 생성 AI를 \n통한 기업 고객의 클라우드 지출 확대를 위해 AI 투자를 지속\n\nn 구글은 수익률이 높은 클라우드 컴퓨팅 시장에서 아마존과 마이크로소프트를 따라잡고자 생성 AI를 \n통한 기업 고객의 클라우드 지출 확대를 위해 AI 투자를 지속  \n∙구글은 앤스로픽 외에도 AI 동영상 제작 도구를 개발하는 런웨이(Runway)와 오픈소스 소프트웨어 \n기업 허깅 페이스(Hugging Face)에도 투자\n∙구글은 챗GPT의 기반 기술과 직접 경쟁할 수 있는 차세대 LLM ‘제미니(Gemini)’를 포함한 자체 AI \n시스템 개발에도 수십억 달러를 투자했으며, 2024년 제미니를 출시할 계획\n☞ 출처 : The Wall Street Journal, Google Commits $2 Billion in Funding to AI Startup Anthropic, 2023.10.27.\nBloomberg, AI Startup Anthropic to Use Google Chips in Expanded Partnership, 2023.11.09.\n\n1. 정책/법제  \n2. 기업/산업 \n3. 기술/연구 \n 4. 인력/교육\n미국 프런티어 모델 포럼, 1,000만 달러 규모의 AI 안전 기금 조성\nn 구글, 앤스로픽, 마이크로소프트, 오픈AI가 참여하는 프런티어 모델 포럼이 자선단체와 함께 AI \n안전 연구를 위한 1,000만 달러 규모의 AI 안전 기금을 조성\nn 프런티어 모델 포럼은 AI 모델의 취약점을 발견하고 검증하는 레드팀 활동을 지원하기 위한 \n모델 평가 기법 개발에 자금을 중점 지원할 계획\nKEY Contents\n£ 프런티어 모델 포럼, 자선단체와 함께 AI 안전 연구를 위한 기금 조성\nn 구글, 앤스로픽, 마이크로소프트, 오픈AI가 출범한 프런티어 모델 포럼이 2023년 10월 25일 AI 안전 \n연구를 위한 기금을 조성한다고 발표\n∙참여사들은 맥거번 재단(Patrick J. McGovern Foundation), 데이비드 앤 루실 패커드 재단(The \nDavid and Lucile Packard Foundation) 등의 자선단체와 함께 AI 안전 연구를 위한 기금에 \n1,000만 달러 이상을 기부 \n∙또한 신기술의 거버넌스와 안전 분야에서 전문성을 갖춘 브루킹스 연구소 출신의 크리스 메서롤(Chris \nMeserole)을 포럼의 상무이사로 임명\nn 최근 AI 기술이 급속히 발전하면서 AI 안전에 관한 연구가 부족한 시점에, 포럼은 이러한 격차를 해소\n하기 위해 AI 안전 기금을 조성\n∙참여사들은 지난 7월 백악관 주재의 AI 안전 서약에서 외부자의 AI 시스템 취약점 발견과 신고를 \n촉진하기로 약속했으며, 약속을 이행하기 위해 기금을 활용해 외부 연구집단의 AI 시스템 평가에 \n자금을 지원할 계획\n£ AI 안전 기금으로 AI 레드팀을 위한 모델 평가 기법 개발을 중점 지원할 계획\nn 프런티어 모델 포럼은 AI 안전 기금을 통해 AI 레드팀 활동을 위한 새로운 모델 평가 기법의 개발을 \n중점 지원할 예정\n\n▹ 구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화 ················································ 11\n   ▹ IDC, 2027년 AI 소프트웨어 매출 2,500억 달러 돌파 전망··········································· 12\n   ▹ 빌 게이츠, AI 에이전트로 인한 컴퓨터 사용의 패러다임 변화 전망································ 13\n   ▹ 유튜브, 2024년부터 AI 생성 콘텐츠 표시 의무화···························································· 14\n 3. 기술/연구\n   ▹ 영국 과학혁신기술부, AI 안전 연구소 설립 발표······························································ 15\n   ▹ 구글 딥마인드, 범용 AI 모델의 기능과 동작에 대한 분류 체계 발표······························ 16\n   ▹ 갈릴레오의 LLM 환각 지수 평가에서 GPT-4가 가장 우수 ··········································· 17\n   \n 4. 인력/교육     \n   ▹ 영국 옥스퍼드 인터넷 연구소, AI 기술자의 임금이 평균 21% 높아······························· 18\n   \n   \n \nⅡ. 주요 행사\n   ▹CES 2024 ····························································································································· 19', 
            additional_kwargs={}, 
            response_metadata={}, 
            name='pdf_search'
        )
    ]
}


In [ ]:
{
    'output': '구글은 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며, 이 중 5억 달러를 우선 투자했습니다. 향후 15억 달러를 추가로 투자할 계획입니다.', 
    'messages': [
        AIMessage(
            content='구글은 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며, 이 중 5억 달러를 우선 투자했습니다. 향후 15억 달러를 추가로 투자할 계획입니다.', 
            additional_kwargs={}, 
            response_metadata={}
        )
    ]
}


In [15]:
from langchain_teddynote.messages import AgentStreamParser
response = agent_with_chat_history.stream(
    {"input": "구글이 앤스로픽에 투자한 금액을 문서에서 찾아줘"},
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "abc123"}},
)
agent_stream_parser = AgentStreamParser()
for step in response:
    # print(step)
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: pdf_search
query: 구글 앤스로픽 투자 금액
Log: 
Invoking: `pdf_search` with `{'query': '구글 앤스로픽 투자 금액'}`



[관찰 내용]
Observation: 1. 정책/법제  
2. 기업/산업 
3. 기술/연구 
 4. 인력/교육
구글, 앤스로픽에 20억 달러 투자로 생성 AI 협력 강화 
n 구글이 앤스로픽에 최대 20억 달러 투자에 합의하고 5억 달러를 우선 투자했으며, 앤스로픽은 
구글과 클라우드 서비스 사용 계약도 체결
n 3대 클라우드 사업자인 구글, 마이크로소프트, 아마존은 차세대 AI 모델의 대표 기업인 
앤스로픽 및 오픈AI와 협력을 확대하는 추세
KEY Contents
£ 구글, 앤스로픽에 최대 20억 달러 투자 합의 및 클라우드 서비스 제공
n 구글이 2023년 10월 27일 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며, 이 중 5억 
달러를 우선 투자하고 향후 15억 달러를 추가로 투자할 방침
∙구글은 2023년 2월 앤스로픽에 이미 5억 5,000만 달러를 투자한 바 있으며, 아마존도 지난 9월 
앤스로픽에 최대 40억 달러의 투자 계획을 공개
∙한편, 2023년 11월 8일 블룸버그 보도에 따르면 앤스로픽은 구글의 클라우드 서비스 사용을 위해 
4년간 30억 달러 규모의 계약을 체결
∙오픈AI 창업자 그룹의 일원이었던 다리오(Dario Amodei)와 다니엘라 아모데이(Daniela Amodei) 
남매가 2021년 설립한 앤스로픽은 챗GPT의 대항마 ‘클로드(Claude)’ LLM을 개발
n 아마존과 구글의 앤스로픽 투자에 앞서, 마이크로소프트는 차세대 AI 모델의 대표 주자인 오픈
AI와 협력을 확대
∙마이크로소프트는 오픈AI에 앞서 투자한 30억 달러에 더해 2023년 1월 추가로 100억 달러를 
투자하기로 하면서 오픈AI의 지분 49%를 확보했으며, 오픈AI는 마이크로소프트의 애저(Azure) 
클라우드 플랫폼을 사용해 AI 모델을 훈련
£ 

In [ ]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input": "이전의 답변을 영어로 번역해 주세요"},
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "abc123"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)


In [ ]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {
        "input": "2024년 프로야구 플레이오프 진출 5개팀을 검색해서 알려주세요. 한글로 답변하세요"
    },
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "abc456"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)


In [ ]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input": "이전의 답변을 SNS 게시글 형태로 100자 내외로 작성하세요."},
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "abc456"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)


In [ ]:
########## 8. 에이전트를 실행하고 결과를 확인합니다. ##########

# 질의에 대한 답변을 출력합니다.
response = agent_with_chat_history.stream(
    {"input": "이전의 답변에 한국 시리즈 일정을 추가하세요."},
    # 세션 ID를 설정합니다.
    # 여기서는 간단한 메모리 내 ChatMessageHistory를 사용하기 때문에 실제로 사용되지 않습니다
    config={"configurable": {"session_id": "abc456"}},
)

for step in response:
    agent_stream_parser.process_agent_steps(step)
